# Práctica 3. Reconocimiento de Entidades Nombradas. NER

Andrés González Flores

Procesamiento de Lenguaje Natural

Facultad de Ingeniería, UNAM

## Objetivo

Realizar un reconocimiento de Entidades Nombradas (NER) a partir de un modelo secuencial y con el corpus ‘ner dataset.csv’ que se proporciona.

## Instrucciones

Se deberán seguir los siguientes pasos:

1. Obtener las sentencias a partir del csv. En este archivo, se indica cada inicio de sentencia con ‘Sentence: n’. Se cuenta con 1000 sentencias que conformarán el corpus de entrenamiento y evaluación.
2. Preprocesar los datos.
3. Separar los datos en corpus de entrenamiento (70%) y corpus de evaluación (30%).
4. Entrenar un modelo secuencial a partir del corpus de entrenamiento. Deberán definirse los hiperparámetros.
5. Evaluar el desempeño del sistema a partir del corpus de evaluación y con la métrica de Exactitud (Accuracy).
6. Ejemplificar el reconocimiento de entidades nombradas con 5 sentencias del corpus de evaluación.

## Dessarrollo

In [1]:
import csv
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import chain
from tqdm.notebook import tqdm as nbtqdm
from pprint import pprint

In [2]:
# Definición de constantes
SEED = 42
CORPUS_PATH = './ner_dataset.csv'
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<unk>'

np.random.seed(SEED)

### Paso 1. Obtener sentencias.


In [3]:
with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    csvlist = list(reader)

In [5]:
sentences = []
sent_i = []
for row in csvlist[1:]:
    if row[0] is not '':
        sentences.append(sent_i)
        sent_i = []
    sent_i.append((row[1], row[2]))
# Mi algoritmo está feo: agrega la primer sentencia vacía y no agrega la última
del sentences[0]
sentences.append(sent_i)

In [6]:
print(f'Número de oraciones = {len(sentences)}\n')
print('Ejemplos:')
print('\n'.join([' '.join([w for w, tag in sent]) for sent in sentences[:5] ]))

Número de oraciones = 1000

Ejemplos:
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "
They marched from the Houses of Parliament to a rally in Hyde Park .
Police put the number of marchers at 10000 while organizers claimed it was 1,00,000 .
The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .


### Paso 2. Preprocesamiento

In [7]:
freq_tokens = Counter([w for sentence in sentences for w,_ in sentence])
freq_tokens.most_common(10)

[('the', 1111),
 ('.', 995),
 (',', 637),
 ('in', 576),
 ('of', 568),
 ('to', 505),
 ('a', 449),
 ('and', 391),
 ('The', 241),
 ("'s", 202)]

In [56]:
# vocab_unk = { for w, x_i in vocab.items() }

9

In [45]:
vocab = { 
    x[0] : i for i, x in enumerate(freq_tokens.most_common())
}
pprint(list(vocab.items())[:10])

[('the', 0),
 ('.', 1),
 (',', 2),
 ('in', 3),
 ('of', 4),
 ('to', 5),
 ('a', 6),
 ('and', 7),
 ('The', 8),
 ("'s", 9)]
